---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [133]:
def date_sorter(dataFrame):
    
    # mm/dd/yyyy
    dates_found = df.str.extractall(r'(?P<date>(?P<month>[\d]{1,2})[/-]+(?P<day>[\d]{1,2})[/-]+(?P<year>\d{4}))')
    date_cols = ["date","day", "month", "year"]
    dates_found = dates_found[date_cols]
    # mm/dd/yy

    remaining_indices = ~df.index.isin([x[0] for x in dates_found.index])
    dates_2 = df[remaining_indices].str.extractall(r'(?P<date>(?P<month>[\d]{1,2})[/-]+(?P<day>[\d]{1,2})[/-]+(?P<year>\d{2}))')
    dates_2["year"] = "19" + dates_2["year"]
    dates_2 = dates_2[date_cols]
    dates_found = dates_found.append(dates_2)
    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
    remaining_indices = ~df.index.isin([x[0] for x in dates_found.index])
    dates_3 = df[remaining_indices].str.extractall(r'(?P<date>(?P<month>[A-Za-z.]+)[\s-]?(?P<day>[\d]{1,2})[,\s-]+?(?P<year>\d{4}))')
    dates_3["month"] = dates_3["month"].str.replace(".", "")
    #dates_3 = dates_3["month"].str.replace("Age", "")
    #dates_3 = dates_3[dates_3["month"] != "Age"]
    dates_3 = dates_3[date_cols]
    dates_found = dates_found.append(dates_3)

    #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    remaining_indices = ~df.index.isin([x[0] for x in dates_found.index])
    dates_4 = df[remaining_indices].str.extractall(r'(?P<date>(?P<day>[\d]{1,2})[\s-]?(?P<month>[A-Za-z.]+)[,\s-]+?(?P<year>\d{4}))')
    dates_4["month"] = dates_4["month"].str.replace(".", "")
    dates_4 = dates_4[date_cols]
    dates_found = dates_found.append(dates_4)
    
    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    remaining_indices = ~df.index.isin([x[0] for x in dates_found.index])
    dates_5 = df[remaining_indices].str.extractall(r'(?P<date>(?P<month>[A-Za-z.]+)[\s-]?(?P<day>[\d]{1,2})[a-z]+[,\s-]+?(?P<year>\d{4}))')
    dates_5 = dates_5[date_cols]
    dates_found = dates_found.append(dates_5)
    #Feb 2009; Sep 2009; Oct 2010 # have to ensure not in indices from before
    remaining_indices = ~df.index.isin([x[0] for x in dates_found.index])
    dates_6 = df[remaining_indices].str.extractall(r'(?P<date>(?P<month>[A-Za-z]{3,})[\s-]?(?P<year>\d{4}))')
    dates_6["day"] = 1
    dates_6 = dates_6[date_cols]
    dates_found = dates_found.append(dates_6)
    
    remaining_indices = ~df.index.isin([x[0] for x in dates_found.index])
    dates_7 = df[remaining_indices].str.extractall(r'(?P<date>(?P<month>\d{1,2})[-/](?P<year>\d{4}))')
    dates_7["day"] = 1
    dates_7 = dates_7[date_cols]
    dates_found = dates_found.append(dates_7)
    
    #2008, 2009
    remaining_indices = ~df.index.isin([x[0] for x in dates_found.index])
    dates_8 = df[remaining_indices].str.extractall(r'(?P<date>(?P<year>\d{4}))')
    dates_8["day"] = 1
    dates_8["month"] = 1
    dates_8 = dates_8[date_cols]
    dates_found = dates_found.append(dates_8)
    month_dictionary = {'AFeb': 2,'Apr':4, 'April':4,'Aug':8, 'August':8, 'Dec':12, 'December':12,
                        'Decemeber': 12, 'Feb': 2, 'February': 2, 'IAug':8, 'Jan':1,'Janaury':1,
                        'January': 1,'Jul':7, 'July':7, 'Jun':6, 'June':6, 'Mar':3,'March':3, 
                        'May':5, 'Nov': 11,'November':11, 'Oct':10, 'October':10, 'Sep':9, 
                        'September':9, 'lNovember': 11,'pOct':10,'sNovember':11, 'sSep':9, 'yAug':8 }
    dates_found.replace({"month":month_dictionary}, inplace = True)
    dates_found["month"] = dates_found["month"].astype(str)
    dates_found["day"] = dates_found["day"].astype(str)

    dates_found["day"] = dates_found["day"].apply(lambda x: "0" + x if len(x) == 1 else x)

    dates_found["month"] = dates_found["month"].apply(lambda x: "0" + x if len(x) == 1 else x)
    correct_months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
    dates_found.month[~dates_found["month"].isin(correct_months) == True] = 1
    dates_found["month"] = dates_found["month"].astype(int)
    dates_found["day"] = dates_found["day"].astype(int)
    dates_found = dates_found[dates_found["day"] < 32]
    dates_found["month"] = dates_found["month"].astype(str)
    dates_found["day"] = dates_found["day"].astype(str)
    dates_found["new_date"] = dates_found["day"] + "/" + dates_found["month"] + "/" + dates_found["year"]
    dates_found["new_date"] = pd.to_datetime(dates_found["new_date"])
    dates_found.sort_values(by = "new_date", inplace = True)
    dates_found_indices = pd.Series(list(dates_found.index.labels))[0]
    return dates_found_indices